# Test of bespoke neural network

Import necessary modules:

In [2]:
from classes.MyModel import MyModel
import torch
import torch.nn as nn
import math as math
from pathlib import Path
from torchviz import make_dot
from time import strftime

from modules.helper_functions_tsp import (find_problem_size,
                                          find_distances_array,
                                          cost_fn_fact, 
                                          cost_fn_tensor, 
                                          hot_start, 
                                          hot_start_list_to_string
                                          )

from modules.config import GRAPH_DIR, VERBOSE
from modules.helper_ML_functions import (find_device, 
                                         get_ready_to_train,
                                         train_model
                                         )

from modules.graph_functions import (plot_sine_activation,
                                     #plot_model_training
                                     )

from classes.MyDataLogger import MyDataLogger, MySubDataLogger

Set up constants:

In [3]:
#LOCATIONS = 8                       #locations in problem
#DECODING_FORMULATION = 'original'   #decoding formulation
#                                    #options: 'original', 'new'
#GRAY = False                        #gray code  
#NUM_EPOCHS = 50                     #number of epochs for training
#VERBOSE = False                     #controls how much output the model produces
#SHOTS = 64                          #size of input tensor.  Reduces randomness

In [4]:
datalogger = MyDataLogger()
my_subid = strftime('%H-%M-%S')
sdl = MySubDataLogger(runid = datalogger.runid)
sdl.update_general_constants_from_config()
sdl.update_ml_constants_from_config()
sdl.quantum = False
sdl.validate_input()

SubDataLogger instantiated.  Run ID = 20250403-11-54-28 - 11-54-28


Read in data, report on data read and validate distance array:

In [5]:
#distance_array, best_dist = find_distances_array(LOCATIONS, print_comments=True)
distance_array, best_dist = find_distances_array(sdl.locations, print_comments=True)

Reading distance data
Data will be read from filename networks\five_d.txt.
It is known that the shortest distance is 19


Evaluate cost function and clear cache:

In [ ]:
#cost_fn = cost_fn_fact(LOCATIONS, distance_array, GRAY, method = DECODING_FORMULATION, verbose=VERBOSE)
cost_fn = cost_fn_fact(sdl.locations, 
                       distance_array, 
                       sdl.gray, 
                       method = sdl.formulation, 
                       )
cost_fn.clear_cache()

TypeError: cost_fn_fact() got an unexpected keyword argument 'verbose'

Check if CUDA is available and set the device

In [ ]:
device = find_device()
print(f"Using device: {device}")

Find problem size:

In [ ]:
#qubits = find_problem_size(LOCATIONS, DECODING_FORMULATION)
#print(f'There are {qubits} qubits needed for {LOCATIONS} locations in the {DECODING_FORMULATION} formulation.')
qubits = find_problem_size(sdl.locations, sdl.formulation)
print(f'There are {qubits} qubits needed for {sdl.locations} locations in the {sdl.formulation} formulation.')

Find a hot start using a lazy classical algorithm and find the quality of the hot start:

In [ ]:
#hot_start_list = hot_start(distance_array, LOCATIONS)
hot_start_list = hot_start(distance_array, sdl.locations)
print(f'The hot start location list is {hot_start_list}')
#bin_hot_start_list = hot_start_list_to_string(hot_start_list, LOCATIONS, GRAY, DECODING_FORMULATION)
bin_hot_start_list = hot_start_list_to_string(hot_start_list, 
                                              sdl.locations, 
                                              sdl.gray, 
                                              sdl.formulation
                                              )
print(f'This is equivalent to a binary list: {bin_hot_start_list}')
bin_hot_start_list_tensor = torch.tensor([bin_hot_start_list])
print(f'bin_hot_start_list_tensor = {bin_hot_start_list_tensor}')
distance_tensor = cost_fn_tensor(bin_hot_start_list_tensor, cost_fn).clone().detach().requires_grad_(True)
print(f'The hot start distance is {float(distance_tensor):.2f}, compared to a best distance of {best_dist:.2f}.')

Set up input as repeated hot start tensor calculated above:

In [ ]:
unrepeated_input = bin_hot_start_list_tensor.float().to(device)
my_input = unrepeated_input.repeat(sdl.shots, 1).requires_grad_(True)

Set up model with gradient required:

In [ ]:

#model = MyModel(qubits, NUM_LAYERS, STD_DEV, cost_fn).to(device)
model = MyModel(qubits, 
                sdl.layers, 
                sdl.std_dev, 
                cost_fn
                ).to(device)
for param in model.parameters():
    param.requires_grad = True

Report on model parameters:

In [ ]:
total_params = sum(
	param.numel() for param in model.parameters()
)
trainable_params = sum(
	p.numel() for p in model.parameters() if p.requires_grad
)

print(f'There are {total_params} parameters in total, of which {trainable_params} are trainable')

for name, param in model.named_parameters():
    print(f"Parameter {name} requires_grad: {param.requires_grad}")

Test the Sine activation function and print out a graph:

Plot the sine activation function:

In [ ]:
plot_sine_activation()

Print out model details including graph

In [ ]:
output = model(my_input)
lowest_cost = float(output)
print(f'Output = {output}')
filename = Path(GRAPH_DIR).joinpath('torchviz')

param_dict = dict(model.named_parameters())
print(param_dict)  # Debugging: print the parameters to ensure they seem reasonable
make_dot(output, params=param_dict).render(filename, format="png")
make_dot(output, params=param_dict)

Set up criterion, optimizer and target ready to train model.

In [ ]:
#target, criterion, optimizer = get_ready_to_train(model, OPTIMIZER, LR, WEIGHT_DECAY, momentum = MOMENTUM)
target, criterion, optimizer = get_ready_to_train(model, 
                                                  sdl.gradient_type, 
                                                  sdl.lr, 
                                                  sdl.weight_decay, 
                                                  momentum = sdl.momentum
                                                  )

Train model and print out results:

In [ ]:
lowest_cost, epoch_lowest_cost, epoch_hist, loss_hist, lowest_history = \
    train_model(sdl.iterations,
                model, 
                my_input, 
                target, 
                criterion,
                optimizer,
                print_results=True
                )

In [ ]:
print(f'The lowest cost found was {lowest_cost:.3f} at epoch {epoch_lowest_cost}.')
print(f'The best known cost is {best_dist:.3f} and the hot start cost was {float(distance_tensor):.3f}.')

Plot loss ratio by epoch:

In [ ]:
#plot_model_training(epoch_hist, loss_hist)
sdl.save_plot()


Print out parameters:

In [ ]:
for layer in model.children():
    if isinstance(layer, nn.Linear):
        print('weight:', layer.weight)
        print('bias:', layer.bias)